## Answer generation using Microsoft Phi-1.5 model

-  Microsoft introduced Phi-1.5, a 1.3 billion parameter LLM, predominantly trained on 30 billion tokens of synthetic "textbook - like" data. The model's performance on common sense reasoning, and question answering benchmarks is impressive, rivaling models with 5-10 times its size. Phi-1.5 demonstrated competencies in multi-step reasoning, elementary coding, and even displayed rudimentary in-context learning abilities.
- We used Yahoo Q&A dataset from HuggingFace (https://huggingface.co/datasets/yahoo_answers_qa).

#### Loading and importing required packages

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-0z8nds64
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-0z8nds64
  Resolved https://github.com/huggingface/transformers.git to commit bd50402b56980ff17e957342ef69bd9b0dd45a7b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8056436 sha256=c58bd7dd78e3256dc59a27e8d20dd4b6f04c26d7a599cfba20e2bb5efb575582
  Stored in directory: /tmp/pip-ephem-wheel-cache-4i6gtbkx/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pickle
from tqdm import tqdm
import time

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Checking access to GPU

In [ ]:
if torch.cuda.is_available():
    print("cuda:0")
    device = 'cuda:0'
else:
    print('cpu')
    device = 'cpu'

cuda:0


#### Reading data from Google Drive

In [ ]:
data_file_path = "/content/drive/My Drive/ANLP/0000.parquet"
data = pd.read_parquet(data_file_path, engine='auto')

In [ ]:
data_ = data.copy()
for i in range(len(data_)):
    if data_.iloc[i, 1][-2] == " ":
        data_.iloc[i, 1] = data_.iloc[i, 1].replace(data_.iloc[i, 1][-2:], "?")
print(data.shape)
data_.head()

(87362, 5)


,id,question,answer,nbestanswers,main_category
0,2020338,Why did the U.S Invade Iraq?,A small group of politicians believed strongly...,[A small group of politicians believed strongl...,News & Events
1,2874684,How to get rid of a beehive?,Call an area apiarist. They should be able to...,[Call an area apiarist. They should be able t...,Education & Reference
2,4193114,Why don't European restaurants serve water?,There's a general belief in Europe (and in fac...,[There's a general belief in Europe (and in fa...,Society & Culture
3,1908421,Why hybrid cars gas mileage is better in city?,hybrid cars save energy in two ways: 1.by stor...,[hybrid cars save energy in two ways: 1.by sto...,Cars & Transportation
4,3608897,Can someone explain the theory of e=mc2?,In general it means that in a very high speed ...,[In general it means that in a very high speed...,Science & Mathematics


#### Pretrained model and tokenizer initialization

In [ ]:
model = AutoModelForCausalLM.from_pretrained('microsoft/phi-1_5', trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-1_5', trust_remote_code=True)

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

#### Generating answers

In [ ]:
# Function for answer generation
def get_answer(question):
    model_input = tokenizer(question, return_tensors='pt', return_attention_mask=False).to(device)
    model_output = model.generate(**model_input, max_length=128)
    answer = tokenizer.batch_decode(model_output)[0]
    return [answer]

In [ ]:
# Questions processing
answers_generated = list()
for i in tqdm(range(67000, 68000)):
    curr_answer = get_answer(data_.iloc[i, 1])[0].replace(data_.iloc[0, 1], '')
    answers_generated.append([curr_answer])

In [ ]:
# Saving obtained answers
with open('answers_67k-68k.pkl', 'wb') as f:
    pickle.dump(answers_generated, f)